In [ ]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup  
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')



In [ ]:
df, code = generate_code_from_db(ctx, qgroup=4028)
df

In [ ]:
pl=sql_todf("""
SELECT UPPER(LTRIM(RTRIM(DisplayValue))) [Selections], PK_PickList [ML_Value], PK_PickListType [PLT]
FROM vwPickLists  
WHERE PK_PickListType > 410
ORDER BY PK_PickListType DESC
""", connstr)  
s=list(pl.loc[pl['PLT']==423]['Selections'])
print(s)
pr = picklist_recommender(ctx.config['connstr'], reset_cache=True, picklist_where=' PK_PickListType > 410 ') 
pr.recommend(s, threshhold=(.825, .5))

#pl['PK_PickListType'].astype('int32')
#l=SQL_INSERT_FROM_DF(pl)

In [ ]:
filepath = r'C:\Users\timko\Downloads\Questionnaire\HVA Network Architecture Questionnaire v0.1.xlsx'


In [ ]:
converts={'Selections':lambda x: re.sub('\d{,2}\.','',x).upper().strip()}
df=pd.read_excel(filepath, converters=converts,  sheet_name='Questionnaire', header=7, usecols='A,B,D,E,F,G').reset_index()
df = df.rename(columns={'ML Score': 'ML_Score','ML Scored': 'ML_Scored' })
df["ID"].fillna(method='ffill', inplace=True) 
df = df.loc[ df["ID"].str.contains('NA') ]   
df.dropna(subset=['ML_Scored'], inplace=True) 
df['ML_Score'] = df['ML_Score'].apply(lambda s: re.sub('[^0-9]','',str(s)))
 
df.loc[df['ID']=='NA22']

In [ ]:

dfp=df.groupby(['ID'], as_index=False).agg({'Selections':list}) 
pr = picklist_recommender(ctx.config['connstr'], reset_cache=False, picklist_where=' PK_PickListType > 410 ') 
dfp['PLT']=dfp['Selections'].apply(lambda s: pr.recommend(str(s))['PK_PicklistType'])
pr.to_cache() 

dff=pd.merge(df,dfp.loc[:,['ID','PLT']], how='left', left_on='ID', right_on='ID')
dff=pd.merge(dff,pl, how='left', left_on=['Selections','PLT'], right_on=['Selections','PLT'])
dff.ML_Value=dff.ML_Value.fillna(0).astype('int32').astype('str')
dff.loc[dff['PLT']==0]

In [ ]:
dff=dff 
dff.SML.fillna(method='ffill', inplace=True)
dff=dff.loc[dff['ML_Scored'] != 0]
dff['sortorder'] = range(1, len(dff)+1)
l=SQL_INSERT_FROM_DF(dff.loc[:, ['ID', 'SML','ML_Score', 'ML_Scored', 'ML_Value', 'sortorder']])
dff.loc[dff['ID']=='NA22']


In [ ]:
sml3 = dff.loc[  (dff['ML_Scored']==1) & (dff['SML']=='SML3') ]

In [ ]:
sml3['ANS']=[f",('{x}','SML3','{y}','x')" for x,y in zip( sml3['ID'] ,sml3['ML_Value'] )]
''.join(list(sml3['ANS']))

In [59]:
base = r"C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\HVA\\2023\\"
pages = {
      '2':'4029','3':'4030','4':'4031','5':'4032','6':'4033' 
} 
for i in pages.keys():
    exts = ['.aspx','.aspx.designer.vb','.aspx.vb']
    for ext in exts:
        with open(f'{base}2023_HVA_NETWORK_1{ext}', 'r', encoding='UTF-8') as fr:
            txt = fr.read()
            txt=txt.replace(f'11',f'{i}{i}')
            txt=txt.replace(f'_1',f'_{i}')
            txt=txt.replace(f'"4028"',f'"{pages[i]}"')
            df, code = generate_code_from_db(ctx, qgroup=pages[i])
            txt=txt.replace(f'<!--controls-->',code)
            #print(txt)
        with open(f'{base}2023_HVA_NETWORK_{i}{ext}', 'w', encoding='UTF-8') as fw:
            fw.write(txt)

<%@ Page Title="" Language="vb" AutoEventWireup="false" MasterPageFile="~/Form.Master" 
CodeBehind="2023_HVA_NETWORK_2.aspx.vb" Inherits="CyberScope._2023_HVA_NETWORK_22" %>
<%@ Register Src="~/CustomControls/CBlabel.ascx" TagName="CBlabel" TagPrefix="uc" %>
<%@ Register Src="~/CustomControls/CBoptionalLinks.ascx" TagName="CBoptionalLinks" TagPrefix="uc" %>
<%@ Register Src="~/CustomControls/CBButtPanel2.ascx" TagName="CBButtPanel" TagPrefix="uc" %>
<%@ Register Src="~/CustomControls/CBtext2.ascx" TagName="CBtext2" TagPrefix="uc" %>
<%@ Register Src="~/CustomControls/CBPickList2.ascx" TagName="CBPickList2" TagPrefix="uc" %>
<%@ Register Src="~/CustomControls/CbPickListMultiSelect.ascx" TagName="CbPickListMultiSelect" TagPrefix="uc" %>
<%@ Register Src="~/CustomControls/CbPickListMultiSelectList.ascx" TagName="CbPickListMultiSelectList" TagPrefix="uc" %>
<%@ Register Src="~/CustomControls/CBYesNo.ascx" TagName="CBYesNo" TagPrefix="uc" %>
<asp:Content ID="Content1" ContentPlaceHolderID="

In [ ]:

ip = issue_provider(ctx)  
ip.driver.quit()
ip.issues

In [ ]:
c=0
for i in range(19000, 19900):
    for j in range(1,4):
        c=c+1
        s=f"INSERT INTO @T VALUES ( {c+i}, '2023-A-Q{j}', {i-18000}, 'SYSTEM-{j}', 'NS')"
        print(s)


In [ ]:
df=sql_todf("""
 SELECT DISTINCT Acronym FROM [Component List] WHERE FK_PK_Component IS NULL AND isActive=1
""", connstr) 
a1 = set(df.Acronym) 

In [ ]:
dff=pd.read_csv(r'C:\Users\Tim\Downloads\agency_and_cve_level.csv') 
a2 = set(dff.acronym)
a2.difference(a1)
# BGSF, EXIM,  UDALL, USIBWC, USITC

In [ ]:
df=pd.read_csv(r'C:\Users\Tim\Downloads\agency_and_cve_level.csv')
df['{cve_id}']=df['cve_id']
df['{acronym}']=df['acronym']
df['{cdm_findings}']=df['cdm_findings']
s=''
for i, r in df.iterrows(): 
    ins=""" INSERT INTO CVEUnremediated (PK_OrgSubmission, PK_CISA_CVE, CDMCountSubmitted, CDMDateSubmitted, SubmissionCount, UserId)
            VALUES( (SELECT MAX(PK_OrgSubmission) FROM vwOrgSubToComponent WHERE PK_ReportingCycle=108 AND Acronym='{acronym}' )
        ,'{cve_id}' ,{cdm_findings}  ,'12-15-2022' ,{cdm_findings} ,2815 ); """
    ins=ins.replace('{cve_id}',r['{cve_id}'])
    ins=ins.replace('{acronym}',r['{acronym}'])
    ins=ins.replace('{cdm_findings}', str(r['{cdm_findings}']) )
    s=s+ins+'\n'
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write(s) 
print(ins)

In [ ]:
df=pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='Sheet1', header=0 ) 
CSP=list(pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='CSP').iloc[:, 0])
ServiceModelType=list(pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='ServiceModelType' ).iloc[:, 0])
Remediated=list(pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='Remediated').iloc[:, 0])
for k in df.columns:
    print(f"d['{k}'] = ''")
lod=[]  
for i in range(1,10):
    d={}
    d['Cloud Service Provider'] = CSP[ random.randint(0, len(CSP)) -1 ]
    d['FedRAMP Package ID'] = f'ID{i}'
    d['Service Type'] = ServiceModelType[ random.randint(0, len(ServiceModelType)) -1 ]
    d['Service Model Type'] = ServiceModelType[ random.randint(0, len(ServiceModelType)) -1 ]
    d['ATO Letter with FedRAMP PMO (Yes or No)'] = 'Yes'
    d['Comments'] = f'Comments{i}'
    lod.append(d)
lod
df=pd.DataFrame(lod) 
write_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx','Sheet1',df)
df=pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='Sheet1', header=0 ) 
df


In [ ]:
ip.driver.quit()

In [ ]:
with open(ctx.get_tempalte_dir()+'sum.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}],SectionNum, SortPos FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO' AND IdText LIKE '4.1._._' ;
""", connstr) 

df['{var}'] = df['{var}'].apply(lambda s : '@SUM_'+re.sub('_[a-z]$','',s))
df['{idt}'] = df['{idt}'].apply(lambda s : f'\'{s}\'') 
df = df.groupby('{var}', as_index=False).agg({'{idt}':list})
df

In [ ]:

df['sql']=[s.replace('{idt}',','.join(x)).replace('{var}',y) for x,y in zip(df['{idt}'], df['{var}'])]
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 
df 

In [ ]:
with open(ctx.get_tempalte_dir()+'frmval.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}], SortPos  FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO'  AND IdText LIKE '2.%' AND Q_TypeID=2 
ORDER BY SortPos;
 
""", connstr) 
df['sql']=[s.replace('{idt}',x).replace('{var}',y) for x,y in zip(df['{idt}'], df['{var}'])]
 
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 



In [ ]:


df['sql']=s
df['sql']=[x.replace('{idt}',y).replace('{var}',z) for x,y,z in zip(df['sql'], df['{idt}'], df['{var}'])]
 
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 
df

In [ ]:
df,code=generate_code_from_db(ctx,4019) 
#generate_section(4018, code)
df[:10]
#sql=df_to_sql(df,ctx,template_name='fsma_QuestionsInsert.sql')

In [ ]:
def generate_section(section, code=''): 
    df=sql_todf("SELECT DISTINCT PK_QuestionGroup, ASPX FROM vwQuestions  WHERE PK_FORM = '2023-Q1-CIO'", connstr).set_index('PK_QuestionGroup' )
    d=df.T.to_dict(orient='records')[0] 
    print(d)
    path=f'C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\{d[section]}'
    with open(path, 'r', encoding='utf-16') as f:
        s=f.read() 
    pname = d[section][d[section].rfind('/')+1:]
    if '<!--questions-->' in s:
        q=re.findall('<!--questions-->.+<!--\/questions-->',s,re.DOTALL)[0]
        s=s.replace(q,f'<!--questions-->\n{code}\n<!--/questions-->')
        with open(f"{ctx.get_dest()}{pname}", 'w', encoding='utf-16') as f:
            f.write(s) 



In [ ]:

def parse_grid(tnum, pk, QCol='Question', PivotCols=4): 
    ip = issue_provider(ctx)
    driver=ip.driver
    driver.get(f"https://dayman.cyber-balance.com/jira/si/jira.issueviews:issue-html/CS-{tnum}/CS-{tnum}.html")   
    ldf = pd.read_html(driver.page_source, match="\d\.\d\.\d", header=1)
    df=ldf[0].iloc[:,:].rename(columns={QCol: "QuestionText"})
 
    df['identifier_text'] = df['QuestionText'].apply(lambda s: re.findall('^[\d\.\w]{1,7}',s)[0])
    stypes = {k: chr(v+97) for v, k in enumerate(list(df.columns[1:4]))} 
    print(stypes)
    df=df.loc[:,['QuestionText','identifier_text']] 
    df['CTRLCODE']='LABEL'
    lod=df.to_dict(orient='records')
    l=[]
    for i,r in enumerate(lod):
        l.append(lod[i])
        for k in stypes.keys():
            id=lod[i]['identifier_text']+'.'+stypes[k]
            l.append({
                'identifier_text':id
                , 'QuestionText': id+' '+k
                , 'CTRLCODE':'CNT'
                })
    df=pd.DataFrame(l)
    df['{PK_Question}'] = range(pk,len(df)+pk)
    df['sortpos'] = range(1,len(df)+1) 
    df['FK_QuestionType'] = df['CTRLCODE'].apply(lambda s: 2 if s=='CNT' else 18)
    df['QuestionText']=df['QuestionText'].apply(lambda s: re.sub('^[\d\.\w]{1,7}','',s, count=1).strip())
    df['{QuestionText}']=df['QuestionText']
    df['{idt}'] = df['identifier_text'].str.replace('.','_')
    with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
        df['sql']=f.read() 
    for i,r in df.iterrows():
        for c in df.columns: 
            df.loc[i,'sql']= re.sub(str(c),str(df.loc[i,c]),df.loc[i,'sql'])  
    sql='\n,'.join(df['sql'])
    with open(f'{ctx.get_dest()}script.sql', 'w') as f:
        f.write(sql)

    gen=script_generator(ctx)
    code=gen.generate(df)
    with open(f'{ctx.get_dest()}script.aspx', 'w') as f:
        f.write(code)
    
    return df,sql,code

# df,sql,code=parse_grid('9295',37400, QCol='Type of Test')
# print(sql)

In [ ]:
ctrl=''
with open(f'{ctx.get_tempalte_dir()}TXT_OPT.aspx', 'r') as f:
    ctrl=f.read().replace('\n','')

def met_provider(s):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)  
    return m[0] if len(m) > 0 else ''

def cell_provider(s, i):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)
    d={0:'a.i',1:'a.ii',2:'b.i',3:'b.ii',4:'c.i',5:'c.ii' } 
    idt=f'{m[0]}.{d[i]}'
    c=ctrl.replace('{idt}',idt.replace('.','_'))
    return f'{c}' if len(m) > 0 else ''

def lbl_provider(s): 
    if type(s) != str : return s
    lbl='<uc:CBlabel ID="CBlabel_{idt}_i" PK_key="{idt}" runat="server" LabelType="Question" />'   
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s) 
    idt=f'{m[0]}'
    c=lbl.replace('{idt}',idt.replace('.','_'))
    return c
def get_ids():
    dff=sql_todf("SELECT replace(idtext, '.','_') id, pk_question  FROM vwQuestions WHERE PK_QuestionGroup=4014", ctx.config['connstr'])
    print(dff.iloc[0])
    ids = {k:v for k, v in zip(dff['id'], dff['pk_question'] )  }
    return ids
ids=get_ids()   
list(ids.keys())[3:5] 
 

In [ ]:
df=pd.read_excel(r'C:\Users\Tim\Documents\CIO\CIO_SECTION_2.xlsx',  sheet_name='All', header=0 ) # Enterprise Identities  # Public Identities 

#df.fillna('', inplace=True)
#df.drop([0], axis=0, inplace=True)
df.rename(columns={'Unnamed: 2':'.','Unnamed: 4':'..','Unnamed: 6':'...'}, inplace=True)
for i,c in enumerate(df.columns[1:]): 
    df.loc[1:,c] = df.loc[1:, 'Question'].apply(lambda s:  cell_provider(s,i)  )

#mets = [str(m)  for m in list(dff['m']) if '2' in str(m) ]  

df['Question']=df['Question'].apply(lbl_provider )
df.to_html(f'{ctx.get_dest()}parsed.html',index=False)
 
s=''
with open(f'{ctx.get_dest()}parsed.html', 'r') as f:
    s=f.read()
s=re.sub('<th>\.{1,3}</th>','',s)
s=re.sub('th>','td>',s) 
s=re.sub('<td>Number','<td colspan="2">Number',s)

s=re.sub('<thead>|</thead>|<tbody>|</tbody>','',s) 
for k in ids:
    s=s.replace(f'PK_key="{k}"',f'PK_key="{str(ids[k])}"' )
    s=s.replace(f'PK_Question="{k}"',f'PK_Question="{str(ids[k])}"' )
with open(f'{ctx.get_dest()}script.html', 'w') as f: 
    s=html.unescape(s) 
    f.write(s)
df[:5]

In [ ]:
dff=df
dff['m']=df['Question'].apply(lambda s:  met_provider(s)  ) 
dff=dff[['Question','m']] 
dff['s']=dff['m'].apply(lambda m: f"'{m}.a.i','{m}.a.ii','{m}.b.i','{m}.b.ii','{m}.c.i','{m}.c.ii'")
lod=[]
pk=36700
sortpos=1
for  i,r in dff[1:].iterrows():
    pk=pk+1
    sortpos=sortpos+1
    m=r['m']
    lod.append({'{idt}':str(m), '{Question}':r['Question'], '{PK}':str(pk), '{sortpos}':str(sortpos), 'FK_QuestionType':'18'})
    idt = f"'{m}.a.i','{m}.a.ii','{m}.b.i','{m}.b.ii','{m}.c.i','{m}.c.ii'".replace('\'','').split(',') 
    for i in idt: 
        sortpos=sortpos+1
        pk=pk+1
        lod.append({'{idt}':str(i), '{Question}':str(i), '{PK}':str(pk), '{sortpos}':str(sortpos), 'FK_QuestionType':'2'})
lod
sql=''
with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
    sql=f.read() 
s=''  
script=''  
for d in lod: 
    s=sql
    for k in d.keys(): 
        s=s.replace(k,d[k])
    script=script+','+s+'\n'
script
with open(f'{ctx.get_dest()}script.sql', 'w') as f:  
    f.write(script)

#print(s)
#dff

In [ ]:


df=sql_todf("SELECT DISTINCT PK_Question [{PK}], IDTEXT [{idt}], REPLACE(QText,'.','.') [{QuestionText}], sortpos [{sortpos}], FK_QuestionType [{FK_QuestionType}] FROM vwQuestions  WHERE PK_FORM = '2023-Q1-CIO' AND PK_QuestionGroup=4014", connstr) 
sql=''
with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
    sql=f.read()  
r, l, pq, pqt=[], [],'','' 
for i,r in df[:].iterrows():
    if len(r['{QuestionText}']) > 20 and pqt != r['{QuestionText}']:
        pqt = r['{QuestionText}']
        #print(pqt) 
    s=sql
    s=s.replace('{QuestionText}',qgen(r, pqt) )
    for c in df.columns:
        s=s.replace(c, str(r[c])) 
        
    l.append(s)  
s='\n,'.join(l)
print(s) 
